In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("data/dataExport.csv").dropna()
df.head()

In [ ]:
path = 'D:\Learning\DataExport'

In [ ]:
corpus = '. '.join(df.iloc[:,2:].melt()['value'].values).lower()
open(path + 'real_estates.txt', 'w', encoding='utf-8').write(corpus)
len(corpus)

In [ ]:
with open(path + '/replace_with_2_spaces.txt', 'r', encoding='utf-8') as f:
    text_need_replace = f.read()
    f.close()

In [ ]:
#removing emojis
import re
def remove_emojis(string):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f" 
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', string)

def remove_link(string):
  return re.sub(r'^https?:\/\/.*[\r\n]*', '', string, flags=re.MULTILINE)

def remove(string):
  lis = ['*', '=', ',', '..', '-', '_', '/', '+', '^', '?', '<', '>', ';', '{', '}', '[', ']', '!', '|', '&', '$', '~', '`', ')', '(']
  for i in lis:
    string = string.replace(i, '')
  return string

def remove_space(string):
  return " ".join(string.split())

def replace_with_2_space(text):
    abbr_to_replace = [txt.split(' ', 1) for txt in text_need_replace.split('\n')]
    for line in abbr_to_replace:
        old = line[0]
        new = line[1]
        text = text.replace(old, new)
        text = text.replace(new, new)
    return text

def solve(string):
  func = [remove_emojis, remove_link, remove, remove_space, replace_with_2_space]
  for i in func:
    string = i(string)
  return string

In [ ]:
corpus = solve(corpus)
open(path+'real_estates_after.txt', 'w', encoding='utf-8').write(corpus)

In [ ]:
pip install -U nltk

In [ ]:

!pip install underthesea nltk

In [ ]:
import pickle
import string
import re
from underthesea import sent_tokenize
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE, Laplace, KneserNeyInterpolated, WittenBellInterpolated
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [ ]:

with open(path + 'real_estates_after.txt', 'r', encoding='utf-8') as f:
    real_estates_corpus = f.read()

Tiếp theo là các bước tiền xử lí cơ bản để lấy đầu vào cho mô hình n-gram:

- Tách đoạn văn ra thành từng câu nhỏ
- Loại bỏ các dấu câu trong câu
- Tách từ (tokenize) với mỗi câu

In [ ]:
def tokenize(sent):
    tokens = word_tokenize(sent)
    table = str.maketrans('', '', string.punctuation)
    tokens = [word.translate(table) for word in tokens]
    tokens = [word for word in tokens if word] # remove empty
    return tokens

In [ ]:
sent_corpus = sent_tokenize(corpus) # tách câu
len(sent_corpus)


In [ ]:
word_corpus = [tokenize(sent) for sent in sent_corpus]

In [ ]:
n = 6

In [ ]:
# tách theo n gram
train_data, padded_sents = padded_everygram_pipeline(n, word_corpus)

In [ ]:
vi_model = KneserNeyInterpolated(n) # tạo mô hình n-gram interpolation
vi_model.fit(train_data, padded_sents)

In [ ]:
def generate_sent(model, max_num_words, pre_words=[]):
    content = pre_words
    for i in range(max_num_words):
        token = model.generate(1, text_seed=content[-(n - 1):])
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return content

In [ ]:

detokenize = TreebankWordDetokenizer().detokenize

In [ ]:
def recommend_real_estates_search(txt_input):
#     txt_input = pipeline_preprocessing_corpus(txt_input)
    tokens = word_tokenize(txt_input)
    output_recommend = []
    for i in range(1, 10):
        output = generate_sent(vi_model, i, tokens.copy())
        output_recommend.append(detokenize(output))
    output_recommend = list(set(output_recommend)) 
    return '\n'.join(output_recommend)

In [ ]:
txt_input = input()
print(recommend_real_estates_search(txt_input))